In [1]:
import numpy as np
from pycoin.ecdsa.secp256k1 import secp256k1_generator as g
myStuId = 922174

In [2]:
x, y = 4*g
print("x = {}\ny = {}".format(hex(x), hex(y)))

x = 0xe493dbf1c10d80f3581e4904930b1404cc6c13900ee0758474fa94abe8c4cd13
y = 0x51ed993ea0d455b75642e2098ea51448d967ae33bfbdfe40cfe97bdc47739922


In [3]:
x, y = 5*g
print("x = {}\ny = {}".format(hex(x), hex(y)))

x = 0x2f8bde4d1a07209355b4a7250a5c5128e88b84bddc619ab7cba8d569b240efe4
y = 0xd8ac222636e5e3d6d4dba9dda6c9c426f788271bab0d6840dca87d3aa6ac62d6


In [4]:
x, y = myStuId*g
print("x = {}\ny = {}".format(hex(x), hex(y)))

x = 0x10e2f439abcaa3e88b3b1bc5dacb7e34996960ad1a4993366e726926c12ad300
y = 0xecdacad629ca781c202c85ea40dd4a686f910a41f637cf474fd9fd5e178083b5


In [5]:
def countAD(oper_lst):
    counter_d = 0
    counter_a = 0
    counter_s = 0
    for x in oper_lst:
        if x == 'd':
            counter_d += 1
        elif x == 's':
            counter_s += 1
        else:
            counter_a += 1
    return counter_d, counter_a, counter_s
def showDetails(myStuId):
    oper_lst = []
    id_copy = myStuId
    while id_copy > 1:
        bin_id = bin(id_copy)
        if bin_id[-1] == '0':
            oper_lst += ['d']
            id_copy >>= 1
        else:
            oper_lst += ['a']
            id_copy -= 1
    d, a, s = countAD(oper_lst)
    print("My student ID in binary = ", bin(myStuId))
    print("# of double = {}\n# of add = {}".format(d, a))
    print("Detailed operations: ", oper_lst)
    return oper_lst
showDetails(myStuId)

My student ID in binary =  0b11100001001000111110
# of double = 19
# of add = 9
Detailed operations:  ['d', 'a', 'd', 'a', 'd', 'a', 'd', 'a', 'd', 'a', 'd', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'd', 'd', 'a', 'd', 'a', 'd']


['d',
 'a',
 'd',
 'a',
 'd',
 'a',
 'd',
 'a',
 'd',
 'a',
 'd',
 'd',
 'd',
 'd',
 'a',
 'd',
 'd',
 'd',
 'a',
 'd',
 'd',
 'd',
 'd',
 'd',
 'a',
 'd',
 'a',
 'd']

In [15]:
def speeeedup(myStuId):
    details = showDetails(myStuId)
    result = []
    beg_lst = []
    end_lst = []
    i = 0
    while i < len(details)-5:
        if details[i] == 'd' and details[i+1] == 'a':
            j = i+2
            while j < len(details) and details[j] == 'd' and details[j+1] == 'a':
                j += 2
            if j-i > 4:
                beg_lst.append(i)
                end_lst.append(j)
            i = j
        else:
            i +=1
    k = 0
    while k < len(details):
        if len(beg_lst) > 0:
            if k == beg_lst[0]:
                result.append('a')
                beg_tmp = beg_lst.pop(0)
                end_tmp = end_lst.pop(0)
                span = (end_tmp-beg_tmp)//2
                for l in range(span):
                    result.append('d')
                result.append('s')
                k = end_tmp
            else:
                result.append(details[k])
                k += 1
        else:
            result.append(details[k])
            k += 1
    d, a, s = countAD(result)
    print("\n########## After Optimization ##########")
    print("# of double = {}\n# of add = {}\n# of subtract = {}".format(d, a, s))
    print("Detailed operations: ", result)
speeeedup(myStuId)

My student ID in binary =  0b11100001001000111110
# of double = 19
# of add = 9
Detailed operations:  ['d', 'a', 'd', 'a', 'd', 'a', 'd', 'a', 'd', 'a', 'd', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'd', 'd', 'a', 'd', 'a', 'd']

########## After Optimization ##########
# of double = 19
# of add = 5
# of subtract = 1
Detailed operations:  ['a', 'd', 'd', 'd', 'd', 'd', 's', 'd', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'd', 'd', 'a', 'd', 'a', 'd']


In [20]:
def modInverse(a, m) : 
    m0 = m 
    y = 0
    x = 1
    if (m == 1) : 
        return 0
    while (a > 1) : 
        q = a // m 
        t = m 
        m = a % m 
        a = t 
        t = y 
        # Update x and y 
        y = x - q * y 
        x = t 
    # Make x positive 
    if (x < 0) : 
        x = x + m0 
    return x 

def signing():
    da = myStuId
    qa = da * g
    n = g.order()
    z = 0x5c77b910317158b406cbfd43f7585f553b24f5967dcb78d9edc02a58007e9218
    print("Hashed: ", hex(z))
    # k and n should be coprimes
    k = 2
    k_inverse = modInverse(k, n)
    x, y = k * g
    r = x % n
    s = (k_inverse * (z + r * da)) % n
    print("Signature pair:\nr = {}\ns = {}".format(hex(r), hex(s)))
    return r, s, n, qa, z
def verification(r, s, n, qa, z):
    if r < 1 or r > n_order:
         print('verification failed, out of range.')
         exit(1)
    elif s < 1 or s > n_order:
         print('verification failed, out of range.')
         exit(1)
    else:
        w = modInverse(s, n)%n
        u1 = z * w % n
        u2 = r * w % n
        x1, y1 = u1 * g + u2 * qa
        if x1 % n == r:
            print("Verification succeeded.")
        else:
            print("Verification failed.")
r, s, n_order, qA, z_msg = signing()
verification(r, s, n_order, qA, z_msg)

Hashed:  0x5c77b910317158b406cbfd43f7585f553b24f5967dcb78d9edc02a58007e9218
Signature pair:
r = 0xc6047f9441ed7d6d3045406e95c07cd85c778e4b8cef3ca7abac09b95c709ee5
s = 0xce51534bdbe0052a0e12c78d4c505aca975e241961ec622fa4a7763aad1f6a5a
Verification succeeded.
